In [1]:
import pandas as pd
from scipy.stats import skew, kurtosis, ttest_ind, f_oneway, pearsonr

# Load data
df = pd.read_csv("/content/Fear data assignment.sav-addmaple-conversion.csv")

# Step 1: Convert Likert scales to numeric
agreement_map = {
    "Strongly disagree": 1,
    "Disagree": 2,
    "Neither agree nor disagree": 3,
    "Agree": 4,
    "Strongly agree": 5
}

frequency_map = {
    "Not at all": 1,
    "Rare, less than a day or two": 2,
    "Several days": 3,
    "More than 7 days": 4,
    "Nearly every day for 2 weeks": 5
}

# Define item groups
fear_items = [
    "I am most afraid of coronavirus-19.",
    "It makes me uncomfortable to think about coronavirus-19.",
    "My hands become clammy when I think about coronavirus-19.",
    "I am afraid of losing my life because of coronavirus-19.",
    "When watching news and stories about coronavirus-19 on social media, I become nervous or anxious",
    "I cannot sleep because I’m worrying about getting coronavirus-19.",
    "My heart races or palpitates when I think about getting coronavirus-19."
]

anxiety_items = [
    "I felt dizzy, lightheaded, or faint, when I read or listened to news about the coronavirus.",
    "I had trouble falling or staying asleep because I was thinking about the coronavirus.",
    "I felt paralyzed or frozen when I thought about or was exposed to information about the coronavirus.",
    "I lost interest in eating when I thought about or was exposed to information about the coronavirus.",
    "I felt nauseous or had stomach problems when I thought about or was exposed to information about the coronavirus."
]

# Map values
for col in fear_items:
    df[col] = df[col].map(agreement_map)

for col in anxiety_items:
    df[col] = df[col].map(frequency_map)

# Compute totals
df["Fear_Total"] = df[fear_items].sum(axis=1)
df["Anxiety_Total"] = df[anxiety_items].sum(axis=1)

# Report template
def report_group(title, desc_vars, score_var):
    print(f"\n{'='*60}\n{title} — Analysis on {score_var}\n{'='*60}")

    # Descriptives
    for var in desc_vars:
        print(f"\n-- Descriptive Statistics for {var} --")
        print(df[var].value_counts())

    print(f"\n-- {score_var} Skewness & Kurtosis --")
    print("Skewness:", skew(df[score_var]))
    print("Kurtosis:", kurtosis(df[score_var]))

    # Parametric tests
    for var in desc_vars:
        print(f"\n-- Parametric Test: {var} vs {score_var} --")
        if df[var].nunique() == 2:
            groups = df[var].unique()
            g1 = df[df[var] == groups[0]][score_var]
            g2 = df[df[var] == groups[1]][score_var]
            t_stat, p_val = ttest_ind(g1, g2)
            print(f"T-test: t = {t_stat:.3f}, p = {p_val:.4f}")
        elif var == "Age":
            corr, p_val = pearsonr(df["Age"], df[score_var])
            print(f"Correlation with Age: r = {corr:.3f}, p = {p_val:.4f}")
        else:
            groups = df.groupby(var)[score_var].apply(list)
            f_stat, p_val = f_oneway(*groups)
            print(f"ANOVA: F = {f_stat:.3f}, p = {p_val:.4f}")

# Group G1
report_group("G1", ["Gender", "Age", "Educational Level"], "Fear_Total")

# Group G2
report_group("G2", ["monthly income", "Marital Status", "Assistance in ADLs"], "Fear_Total")

# Group G3
report_group("G3", ["Hypertension", "Living Alone", "Insomnia"], "Anxiety_Total")

# Group G4
report_group("G4", ["DM", "Angina", "MI"], "Anxiety_Total")



G1 — Analysis on Fear_Total

-- Descriptive Statistics for Gender --
Gender
male      130
female    120
Name: count, dtype: int64

-- Descriptive Statistics for Age --
Age
47    10
53     9
62     9
48     8
52     8
      ..
34     1
20     1
19     1
23     1
38     1
Name: count, Length: 66, dtype: int64

-- Descriptive Statistics for Educational Level --
Educational Level
low education     141
high education     72
illitrate          37
Name: count, dtype: int64

-- Fear_Total Skewness & Kurtosis --
Skewness: -0.1062649330177961
Kurtosis: -1.1079087464571462

-- Parametric Test: Gender vs Fear_Total --
T-test: t = -1.981, p = 0.0486

-- Parametric Test: Age vs Fear_Total --
Correlation with Age: r = -0.035, p = 0.5838

-- Parametric Test: Educational Level vs Fear_Total --
ANOVA: F = 0.074, p = 0.9288

G2 — Analysis on Fear_Total

-- Descriptive Statistics for monthly income --
monthly income
Low Income         143
Moderate Income     65
High Income         42
Name: count, dtype: 

In [4]:
!pip install fpdf
from fpdf import FPDF

# Create the PDF class
class PDFReport(FPDF):
    def header(self):
        self.set_font("Arial", "B", 14)
        self.cell(0, 10, "Fear and Anxiety Analysis Report", ln=True, align="C")
        self.ln(10)

    def chapter_title(self, title):
        self.set_font("Arial", "B", 12)
        self.cell(0, 10, title, ln=True)
        self.ln(2)

    def chapter_body(self, body):
        self.set_font("Arial", "", 11)
        self.multi_cell(0, 8, body)
        self.ln()

# Initialize PDF
pdf = PDFReport()
pdf.add_page()

# G1–G4 Analysis Content
sections = {
    "G1 — Analysis on Fear_Total": """Descriptive Statistics for Gender:
- Male: 130
- Female: 120

Descriptive Statistics for Age:
(Most frequent values shown)
- 47 (10), 53 (9), 62 (9), 48 (8), 52 (8)

Educational Level:
- Low education: 141
- High education: 72
- Illiterate: 37

Fear_Total:
- Skewness: -0.11
- Kurtosis: -1.11

Parametric Tests:
- Gender: t = -1.981, p = 0.0486 → Significant
- Age: r = -0.035, p = 0.5838 → Not significant
- Educational Level: F = 0.074, p = 0.9288 → Not significant""",

    "G2 — Analysis on Fear_Total": """Monthly Income:
- Low: 143
- Moderate: 65
- High: 42

Marital Status:
- Married: 189
- Not Married: 61

Assistance in ADLs:
- Yes: 105
- No: 145

Fear_Total:
- Skewness: -0.11
- Kurtosis: -1.11

Parametric Tests:
- Income: F = 0.265, p = 0.7675 → Not significant
- Marital Status: t = -0.136, p = 0.8918 → Not significant
- ADLs: t = 0.150, p = 0.8810 → Not significant""",

    "G3 — Analysis on Anxiety_Total": """Hypertension:
- Yes: 163
- No: 87

Living Alone:
- Yes: 17
- No: 233

Insomnia:
- Yes: 122
- No: 128

Anxiety_Total:
- Skewness: 0.69
- Kurtosis: 0.85

Parametric Tests:
- Hypertension: t = -1.488, p = 0.1381 → Not significant
- Living Alone: t = -0.608, p = 0.5435 → Not significant
- Insomnia: t = 0.322, p = 0.7478 → Not significant""",

    "G4 — Analysis on Anxiety_Total": """DM:
- Yes: 124
- No: 126

Angina:
- Yes: 82
- No: 168

MI:
- Yes: 73
- No: 177

Anxiety_Total:
- Skewness: 0.69
- Kurtosis: 0.85

Parametric Tests:
- DM: t = -0.905, p = 0.3661 → Not significant
- Angina: t = -1.485, p = 0.1388 → Not significant
- MI: t = -1.060, p = 0.2902 → Not significant"""
}

# Add content to PDF
for title, body in sections.items():
    pdf.chapter_title(title)
    pdf.chapter_body(body)

# Save to file
pdf.output("Fear_Anxiety_Analysis_Report.pdf")


  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=906212fc76eb82c3f77e68dcd3e2c40a70bc1ae6fe37f1b02d161b9a3e6068b0
  Stored in directory: /root/.cache/pip/wheels/65/4f/66/bbda9866da446a72e206d6484cd97381cbc7859a7068541c36
Successfully built fpdf


UnicodeEncodeError: 'latin-1' codec can't encode character '\u2014' in position 133: ordinal not in range(256)